<a href="https://colab.research.google.com/github/alessandrotofani/Tesi_magistrale/blob/master/6_Federated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Overview: https://www.tensorflow.org/federated

Image classification tutorial: https://www.tensorflow.org/federated/tutorials/federated_learning_for_image_classification

# Installation

In [1]:
!pip install --quiet tensorflow==2.3.0
!pip install --quiet tensorflow_federated==0.17.0
!pip install --quiet --upgrade nest_asyncio

In [2]:
import nest_asyncio
nest_asyncio.apply()
%load_ext tensorboard

In [3]:
import collections
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import pandas as pd 
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import sys 
sys.path.append('/content/drive/MyDrive/Tesi_magistrale/Tesi_magistrale')
import mf

In [5]:
data = pd.read_csv('/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/data.csv')
data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [6]:
data = mf.feature_engineering(data)
# data = mf.feature_scaling(data)
data = pd.get_dummies(data)

In [7]:
shuffled = data.sample(frac=1)
result = np.array_split(shuffled, 5)  
# result

In [8]:
res = []
for dataset in result:
  res.append(mf.feature_scaling(dataset))
# res

In [9]:
# dataset = []
# for subset in res:
#   target = subset.pop('isFraud')
#   # https://www.tensorflow.org/api_docs/python/tf/convert_to_tensor
#   dataset.append(tf.data.Dataset.from_tensor_slices((dict(subset), target.values)))

In [10]:
features = mf.get_col(res[0])
len(features)

1103

In [11]:
# https://medium.com/when-i-work-data/converting-a-pandas-dataframe-into-a-tensorflow-dataset-752f3783c168
dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(res[0][features].values, tf.float32),
            tf.cast(res[0]['isFraud'].values, tf.int32)
        )
    )
)

In [16]:
dataset_ordered = collections.OrderedDict(dataset)

TypeError: ignored

In [13]:
NUM_CLIENTS = 5
NUM_EPOCHS = 10
BATCH_SIZE = 100
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER = 10

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
    return collections.OrderedDict(
       x=tf.reshape(element, [-1, 1103]),
       y=tf.reshape(target, [-1, 1]))
        # element)

  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
      BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

preprocessed_example_dataset = preprocess(dataset)

sample_batch = tf.nest.map_structure(lambda x: x.numpy(),
                                     next(iter(preprocessed_example_dataset)))

sample_batch

TypeError: ignored

In [44]:
federated_data = []
for subset in dataset:
  federated_data.append(preprocess(subset))

In [ ]:
federated_data[0].element_spec

In [46]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, Input

def create_keras_model():
  model = Sequential()
  model.add(Input(shape=(1103,))) 
  # model.add(Dense(1024, activation='relu')) 
  # model.add(Dropout(0.2)) 
  model.add(Dense(512, activation='relu')) 
  model.add(Dropout(0.2)) 
  model.add(Dense(256, activation='relu')) 
  model.add(Dropout(0.2))
  model.add(Dense(128, activation='relu')) 
  model.add(Dropout(0.2))
  model.add(Dense(24, activation='relu')) 
  model.add(Dropout(0.2))
  model.add(Dense(4, activation='relu')) 
  model.add(Dropout(0.2))
  model.add(Dense(1, activation='sigmoid'))
  return model

def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=preprocessed_example_dataset.element_spec,
      loss=tf.keras.losses.BinaryCrossentropy(),
      metrics=[tf.keras.metrics.BinaryAccuracy()])
  
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

state = iterative_process.initialize()

state, metrics = iterative_process.next(state, federated_train_data)
print('round  1, metrics={}'.format(metrics))

TypeError: ignored